# Zillow

## Exploratory Analysis of Zillow Data

In [4]:
import numpy as np
import pandas as pd

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:.2f}'.format

# Split 
from sklearn.model_selection import train_test_split

# Scale
from sklearn.preprocessing import MinMaxScaler

# Stats
import scipy.stats as stats

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

#My Files
import env
from wrangle_zillow import *

In [5]:
df = wrangle_zillow()
df.head()

,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,rawcensustractandblock,...,roomcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,logerror,transactiondate,county
0,14297519,3.50,4.00,6.00,3100.00,6059.00,33634931.00,-117869207.00,4506.00,60590630.07,...,0.00,1998.00,485713.00,1023282.00,2016.00,537569.00,11013.72,0.03,2017-01-01,Orange
1,17052889,1.00,2.00,6.00,1465.00,6111.00,34449266.00,-119281531.00,12647.00,61110010.02,...,5.00,1967.00,88000.00,464000.00,2016.00,376000.00,5672.48,0.06,2017-01-01,Ventura
2,14186244,2.00,3.00,6.00,1243.00,6059.00,33886168.00,-117823170.00,8432.00,60590218.02,...,6.00,1962.00,85289.00,564778.00,2016.00,479489.00,6488.30,0.01,2017-01-01,Orange
3,12177905,3.00,4.00,8.00,2376.00,6037.00,34245180.00,-118240722.00,13038.00,60373001.00,...,0.00,1970.00,108918.00,145143.00,2016.00,36225.00,1777.51,-0.10,2017-01-01,Los Angeles
6,12095076,3.00,4.00,9.00,2962.00,6037.00,34145202.00,-118179824.00,63000.00,60374608.00,...,0.00,1950.00,276684.00,773303.00,2016.00,496619.00,9516.26,-0.00,2017-01-01,Los Angeles


In [6]:
train, validate, test = split_data(df)

In [7]:
scaler, train_scaled, validate_scaled, test_scaled = min_max_scaler(train, validate, test)

In [8]:
train_scaled.head()

,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,rawcensustractandblock,...,roomcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,logerror,transactiondate,county
61675,0.01,0.00,0.09,0.27,0.06,0.00,0.45,0.69,0.00,0.01,...,0.00,0.45,0.01,0.02,0.00,0.01,0.03,0.58,2017-07-28,Los Angeles
7581,0.01,0.11,0.18,0.64,0.13,0.00,0.30,0.58,0.00,0.01,...,0.00,0.58,0.04,0.11,0.00,0.09,0.11,0.63,2017-02-02,Los Angeles
48630,0.01,0.00,0.18,0.27,0.08,0.00,0.47,0.80,0.00,0.00,...,0.00,0.54,0.03,0.07,0.00,0.06,0.07,0.58,2017-06-21,Los Angeles
66151,0.04,0.11,0.27,0.45,0.10,1.00,0.61,0.24,0.00,1.00,...,0.50,0.60,0.01,0.01,0.00,0.00,0.01,0.58,2017-08-14,Ventura
38789,0.02,0.17,0.27,0.45,0.21,0.30,0.24,0.89,0.00,0.30,...,0.00,0.91,0.11,0.16,0.00,0.09,0.19,0.58,2017-05-23,Orange


1. Ask at least 5 questions about the data, keeping in mind that your target variable is logerror. e.g. Is logerror significantly different for properties in LA County vs Orange County vs Ventura County?

target = logerror

log_error = difference of log(Zestimate) and log(SalePrice)

Q: Why did Zillow pick the log error instead of an absolute error metric such as RMSE?

Home sale prices have a right skewed distribution and are also strongly heteroscedastic, so we need to use a relative error metric instead of an absolute metric to ensure valuation models are not biased towards expensive homes. A relative error metric like the percentage error or log ratio error avoids these problems. While we report Zestimate errors in terms of percentages on Zillow.com because we believe that to be a more intuitive metric for consumers, we do not advocate using percentage error to evaluate models in Zillow Prize, as it may lead to biased models The log error is free of this bias problem and when using the natural logarithm, errors close to 1 approximate percentage errors quite closely. See this paper for more on relative errors and why log error should be used instead of percentage error.

https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2635088

2. Answer those questions through a mix of statistical tests and visualizations.

Bonus:

Compute the mean(logerror) by zipcode and the overall mean(logerror). Write a loop that will run a t-test between the overall mean and the mean for each zip code. We want to identify the zip codes where the error is significantly higher or lower than the expected error.